## Introduction - How to use OpenWeather API

<img src="images/OpenWeather_API_Logo.jpg" width="600">


## Query Data - Making calls to the API
When you type www.google.de into your web browser and press enter, your browser will send a request to the server that is hosting google's website, asking to retrieve the contents of the website so it can display it to you. Getting data from APIs works in a similar way. A request is sent to the address of the API, but instead of returning html and javascript files that will be interpreted and rendered by you browser, almost always a JSON file with data is returned. Also while you could use a web browser to send the request, it's not needed and more convenient to use a programming language like Python to do it.

In this notebook we are going to use the requests library. It is one of the most downloaded Python packages today, pulling in around 14M downloads / week. You can find out more about it here: https://pypi.org/project/requests/.
Complete the code below and import the requests package.

In this notebook we will be finally using the OpenWeather API https://openweathermap.org/ and enrich our existing flights data with additional weather data.

In fact, you're going to:

1. Use your API key to make your first call to the OpenWeather API
2. Learn how to adjust your API calls to get the data you need
3. Learn how to access and extract data from your JSON
4. Learn how to flatten nested JSON data and transform it into a DataFrame for future analysis
5. Learn how to make multiple calls to the API with different parameters in an automated way

In [1]:
# Import requests package
import requests

Next, as mentioned above the URL of the API has to be specified. In this example we are going to pull current weather data. The documentation for this API can be found here: https://openweathermap.org/current

The url shown below is what needs to be used in order to connect to the API.

We can see that the url has 
* a fixed part: http://api.openweathermap.org/data/2.5/weather?'
* and a variable part: 'q={city name},{state code},{country_code}&appid={API key}'

<img src="images/Current_Weather_API_Call.png" width="600">

Let's take the fixed part to define a url variable.

In [2]:
# Set URL as url
url = 'http://api.openweathermap.org/data/2.5/weather?'

While the fixed part stays constant, the variable part consists of query strings or parameters, some optional some mandatory, which can be used to select or filter data or define the format or unit of measurement of the response.

Below are the optional and required parameters when pulling current weather data by city name.

<img src="images/Current_Weather_API_Call_Parameters.png" width="600">

Current weather data can be pulled not only by city name but also by

* city ID
* geographic coordinates (latitude, longitude)
* ZIP code
* cities within a rectangular zone
* cities in circle

Depending on which of the listed methods will be used, the number and names of available parameters can change. Read the documention carefully to avoid running into error messages.

In the following example we will stick with pulling the data by city name.

But first:  
We need to define a variable that includes our API key that will be used for the appid parameter. Why? Because we need to authenticate ourselves to the API when we send a request, so the API knows that we have a registered account and can notify us in case we reach our query limit.

To do this,

1. Go to you user profile and go to "My API keys" (https://home.openweathermap.org/api_keys)
2. Copy your API key.
3. Add a variable called "openweather_api_key" to your .env file and put your API key in there. Don't forget to save it!

In [3]:
# To be able to pull the API key from the .env, we need to import our sql module first.
import sql

As the final step before we can do our first API call, we have to put all necessary parameters we chose (city, metric and our API key) into a dictionary. We call it "parameters", but you can choose any name.

In [14]:
parameters = {
    'appid': os.getenv('ow_key'),
    'q': 'Buxtehude',
    'units': 'metric'}

We're almost ready to make our first API call!

One note: Imagine you're having multiple API calls in your script and you want to change the location parameter for all of them to the same variable at the same time. All we need to do is change the value in the dictionary and the API calls will be adjusted. This is much more convenient and less error prone compared to having to find and adjust each individual API call. This approach will also be helpful when we want to make multiple calls for f.e. different locations in one code block using loops, which we will be doing later.

Now that we have the url and the parameters, we can finally send our first API request. Since we want to get data, we need to send a GET request. This is done using the get() function from the requests function. The get() function will send a request and retrieve the response for us.

In [15]:
# Send get request
r = requests.get(url, parameters)

# Print r
print(r)

<Response [200]>


Weirdly, when we print the response, all we see is a code 200 message and no data. 

Did we do something wrong? No! 

Code 200 actually means that our request was successfull. Check out https://en.wikipedia.org/wiki/List_of_HTTP_status_codes to see a list of reponse codes and their definition. Knowing what the different response codes mean will help you understand error messages and resolve issues faster in the future.

So where is the data? The response that the API sent back is actually an object containing multiple elements such as cookies, encoding, headers and content. Run the code blocks below and check out the different elements of the response.

In [ ]:
# Print headers
r.headers

In [7]:
# Print cookies
r.cookies

<RequestsCookieJar[]>

In [8]:
# Print encoding
r.encoding

'utf-8'

In [16]:
# Print content
r.content

b'{"coord":{"lon":9.7,"lat":53.45},"weather":[{"id":800,"main":"Clear","description":"clear sky","icon":"01n"}],"base":"stations","main":{"temp":6.88,"feels_like":5.07,"temp_min":4.71,"temp_max":7.88,"pressure":1024,"humidity":57},"visibility":10000,"wind":{"speed":2.57,"deg":120},"clouds":{"all":0},"dt":1646760769,"sys":{"type":2,"id":2037868,"country":"DE","sunrise":1646718792,"sunset":1646759468},"timezone":3600,"id":2940451,"name":"Buxtehude","cod":200}'

Found it! The content body holds the data. Since by default it's in JSON format, we can use the built-in JSON decoder to increase readability and prepare the data for future data manipulation.

In [17]:
# Apply JSON decoder and save in weather_hh
weather_hh = r.json()

# Print weather_hh
print(weather_hh)

{'coord': {'lon': 9.7, 'lat': 53.45}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 6.88, 'feels_like': 5.07, 'temp_min': 4.71, 'temp_max': 7.88, 'pressure': 1024, 'humidity': 57}, 'visibility': 10000, 'wind': {'speed': 2.57, 'deg': 120}, 'clouds': {'all': 0}, 'dt': 1646760769, 'sys': {'type': 2, 'id': 2037868, 'country': 'DE', 'sunrise': 1646718792, 'sunset': 1646759468}, 'timezone': 3600, 'id': 2940451, 'name': 'Buxtehude', 'cod': 200}


This looks good already, but we can make it look even better! To do this, we are going to use the json Python library and its functions to transform the raw data into a json object and print it with proper indentation.

In [19]:
# Import json package
import json

# Create json object: json_object
json_object = json.loads(r.content)

# Print json_object
json.dumps(json_object, indent = 3)

'{\n   "coord": {\n      "lon": 9.7,\n      "lat": 53.45\n   },\n   "weather": [\n      {\n         "id": 800,\n         "main": "Clear",\n         "description": "clear sky",\n         "icon": "01n"\n      }\n   ],\n   "base": "stations",\n   "main": {\n      "temp": 6.88,\n      "feels_like": 5.07,\n      "temp_min": 4.71,\n      "temp_max": 7.88,\n      "pressure": 1024,\n      "humidity": 57\n   },\n   "visibility": 10000,\n   "wind": {\n      "speed": 2.57,\n      "deg": 120\n   },\n   "clouds": {\n      "all": 0\n   },\n   "dt": 1646760769,\n   "sys": {\n      "type": 2,\n      "id": 2037868,\n      "country": "DE",\n      "sunrise": 1646718792,\n      "sunset": 1646759468\n   },\n   "timezone": 3600,\n   "id": 2940451,\n   "name": "Buxtehude",\n   "cod": 200\n}'

Awesome! Now it's super easy to identify the different key-value pairs and assess the available information.

You might be saying: "Yeah, this is cool and all, but I'm not a weather expert, what do all these fields mean?" 

-> **Read the documention!**

Go to https://openweathermap.org/current#current_JSON and you will find a list of all fields and their definition that can be found in our JSON response.

As mentioned above, we can see the different key-value pairs inside the JSON output. You've probably heard the term key-value pairs before. Do you remember where? Dictionaries, exactly! If you need a quick refresh on what dictionaries are and what they do, check out the official Python documentation here: https://docs.Python.org/3/tutorial/datastructures.html#dictionaries.

Printing the type of our weather_hh variable, which is the API response decoded by the JSON decoder, we get the confirmation that we are working with a dictionary.

In [20]:
# Print type of weather_hh
type(weather_hh)

dict

We can use different techniques to access all or only specific parts of the data. One way is to loop through the dictionary and print all key, value pairs.  
Complete the code below and print all key value pairs.

In [21]:
# Loop through and print all key-value pairs
for key, value in weather_hh.items():
    print(key + ':', value)

coord: {'lon': 9.7, 'lat': 53.45}
weather: [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]
base: stations
main: {'temp': 6.88, 'feels_like': 5.07, 'temp_min': 4.71, 'temp_max': 7.88, 'pressure': 1024, 'humidity': 57}
visibility: 10000
wind: {'speed': 2.57, 'deg': 120}
clouds: {'all': 0}
dt: 1646760769
sys: {'type': 2, 'id': 2037868, 'country': 'DE', 'sunrise': 1646718792, 'sunset': 1646759468}
timezone: 3600
id: 2940451
name: Buxtehude
cod: 200


Alternatively, we can print specific values by referencing their respective key.

In [25]:
# Print values in base
print(weather_hh['base'])

# Print values in main
print(weather_hh['main'])

# Print values in temp
print(weather_hh['main']['temp'])

stations
{'temp': 6.88, 'feels_like': 5.07, 'temp_min': 4.71, 'temp_max': 7.88, 'pressure': 1024, 'humidity': 57}
6.88


## Working with (nested) JSONs
Now that you've learned how to pull current weather data from the OpenWeather API and explore the output, it's time to convert it to a DataFrame. Having the data in a DataFrame allows us to perform data exploration, manipulation or visualization.

The DataFrame() function in the pandas package allows us to transform a dictionary into a DataFrame.
Run the code below to import the library and transform the current weather in Hamburg into a DataFrame.

In [ ]:
# Import pandas package
import pandas as pd

# Transform dictionary to dataframe (will throw an error)
pd.DataFrame(weather_hh)

Weird, we're getting an error message. Do you have an idea what could be the issue?

Don't worry if you have no clue what the problem is. Let's look at our JSON output again. Execute the code below.

In [27]:
# Print content of json_object
json.dumps(json_object, indent = 3)

'{\n   "coord": {\n      "lon": 9.7,\n      "lat": 53.45\n   },\n   "weather": [\n      {\n         "id": 800,\n         "main": "Clear",\n         "description": "clear sky",\n         "icon": "01n"\n      }\n   ],\n   "base": "stations",\n   "main": {\n      "temp": 6.88,\n      "feels_like": 5.07,\n      "temp_min": 4.71,\n      "temp_max": 7.88,\n      "pressure": 1024,\n      "humidity": 57\n   },\n   "visibility": 10000,\n   "wind": {\n      "speed": 2.57,\n      "deg": 120\n   },\n   "clouds": {\n      "all": 0\n   },\n   "dt": 1646760769,\n   "sys": {\n      "type": 2,\n      "id": 2037868,\n      "country": "DE",\n      "sunrise": 1646718792,\n      "sunset": 1646759468\n   },\n   "timezone": 3600,\n   "id": 2940451,\n   "name": "Buxtehude",\n   "cod": 200\n}'

We can see that some key-value pairs have further key-value pairs as values. The key "weather" for example has four key value pairs nested within: "id", "main", "description", "icon". The DataFrame() function expects dictionaries with only one level of key-value pairs, but since we have nested pairs, it throws an error.

There are multiple solutions to this problem and the one that will be the most useful to you depends on what parts of the data you ultimately need for your analysis. We need all data in our output and therefore the method we are going to use is to flatten the JSON data using the json_normalize() function from the pandas package. It flattens our JSON data and transforms it into a DataFrame. Execute the code below and check the output.

In [28]:
# Flatten json and save in weather_hh_norm
weather_hh_norm = pd.json_normalize(weather_hh, sep="_")

# Print weather_hh_norm
weather_hh_norm

,weather,base,visibility,dt,timezone,id,name,cod,coord_lon,coord_lat,...,main_pressure,main_humidity,wind_speed,wind_deg,clouds_all,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1646760769,3600,2940451,Buxtehude,200,9.7,53.45,...,1024,57,2.57,120,0,2,2037868,DE,1646718792,1646759468


Awesome! Now we have a DaraFrame with 1 row and 24 columns. But wait, for some reason the "weather" column still contains data in JSON format. This is because the nested key-value pairs are in a list. Probably because sometimes the weather column can be nested further. In order to resolve this we need to use the advanced parameters of the json_normalize() function.

* record_path = specify the key that is nested deeper
* meta = specify the structure of the remaining JSON
* record_prefix = adds a prefix to the column names in the record_path to avoid duplicate column names

Execute the code below and compare the output to the output above.

In [31]:
# Advanced flattening with json_normalize()
weather_hh_df = pd.json_normalize(weather_hh, 
                                  sep="_", 
                                  record_path="weather", 
                                  meta=[["coord", "lon"], 
                                        ["coord", "lat"], 
                                        "base",
                                        ["main", "temp"],
                                        ["main", "feels_like"],
                                        ["main", "temp_min"],
                                        ["main", "temp_max"], 
                                        ["main", "pressure"], 
                                        ["main", "humidity"], 
                                        "visibility", 
                                        ["wind", "speed"], 
                                        ["wind", "deg"], 
                                        ["clouds", "all"], 
                                        "dt", 
                                        ["sys", "type"], 
                                        ["sys", "id"],
                                        ["sys", "country"],
                                        ["sys", "sunrise"],
                                        ["sys", "sunset"],
                                        "timezone",
                                        "id",
                                        "name",
                                        "cod"], 
                                  record_prefix="weather_")
# Print weather_hh_df
print(weather_hh_df)

   weather_id weather_main weather_description weather_icon coord_lon  \
0         800        Clear           clear sky          01n       9.7   

  coord_lat      base main_temp main_feels_like main_temp_min  ...  \
0     53.45  stations      6.88            5.07          4.71  ...   

           dt sys_type   sys_id sys_country sys_sunrise  sys_sunset timezone  \
0  1646760769        2  2037868          DE  1646718792  1646759468     3600   

        id       name  cod  
0  2940451  Buxtehude  200  

[1 rows x 27 columns]


You're probably thinking: "Wow, this requires a lot of manual work typing all those keys in the paramaters". And yes, you're right. Unfortunately this is one of the easier methods to solve the issue. Also, it's not perfect and comes with its own limitations. If you have data that is nested even deeper you often have no other choice but to write your own flattening function. Fortunately there are other Python specialists out there who have done so and shared their work here: 

* https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
* https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame
* https://medium.com/swlh/converting-nested-json-structures-to-pandas-dataframes-e8106c59976e


Alright, now that the data is in the right format, we don't have to worry about nested JSONs anymore. To make our data more interesting, since right now we only have one row, let's pull current weather data from more cities and create a proper table with multiple rows. In order to do that, we're going to combine, extend and apply everything we've learnt so far:

* Define the parameters in the request URL
* Send a request and retrieve the response using the get() function
* Decode the response using the JSON decoder .json()
* Flatten the JSON file and transform it into a DataFrame

The last step will be to combine all dataframes into one final table.

This time we want to get the current weather data of multiple locations. Unfortunately the location parameter "q" only takes one location at a time. Using a for-loop we should be able to make multiple API calls while iterating through several locations.

First, let's collect all the pieces we need below and

1. Change the location variable to locations and have to include a list with multiple locations
2. Define an empty DataFrame variable
3. Add a for-loop that iterates through all locations, flattens the JSON output and appends it to the DataFrame

**IMPORTANT: Don't run the code below too often per minute, since you only have 60 API calls per minute you can quickly run into a temporary query limit!**

In [38]:
# Set the url
url = 'http://api.openweathermap.org/data/2.5/weather?'

# Set locations
locations = ["q=Hamburg", 
             "q=Berlin", 
             "q=London", 
             "q=Madrid",
             "q=New York",
             "q=Moscow",
             "q=New York",
             "q=Ankara",
             "q=Baghdad",
             "q=Kabul",
             "q=Tokio",
             "q=Taipei",
             "q=Manila",
             "q=Auckland",
             "q=Seoul"]

# Set unit of measurement
unit = "&units=metric"

# Set API key (defined already)
api_key = '&appid=' + os.getenv('ow_key')

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])

# Loop through all locations
for location in locations:
    # Create final url
    url_f = url + location + unit + api_key
    
    # Request data from url
    r = requests.get(url_f)
    
    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = r.json()

    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="weather", 
                                        meta=[["coord", "lon"], 
                                              ["coord", "lat"], 
                                              "base",
                                              ["main", "temp"],
                                              ["main", "feels_like"],
                                              ["main", "temp_min"],
                                              ["main", "temp_max"], 
                                              ["main", "pressure"], 
                                              ["main", "humidity"], 
                                              "visibility", 
                                              ["wind", "speed"], 
                                              ["wind", "deg"], 
                                              ["clouds", "all"], 
                                              "dt", 
                                              ["sys", "type"], 
                                              ["sys", "id"],
                                              ["sys", "country"],
                                              ["sys", "sunrise"],
                                              ["sys", "sunset"],
                                              "timezone",
                                              "id",
                                              "name",
                                              "cod"], 
                                        record_prefix="weather_",
                                        errors='ignore')
    
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df

,weather_id,weather_main,weather_description,weather_icon,coord_lon,coord_lat,base,main_temp,main_feels_like,main_temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod
0,800,Clear,clear sky,01n,10,53.55,stations,7.38,5.67,5.61,...,1646761228,1,1263,DE,1646718725,1646759391,3600,2911298,Hamburg,200
1,800,Clear,clear sky,01n,13.4105,52.5244,stations,5.35,3.54,3.41,...,1646761141,2,2011538,DE,1646717857,1646758622,3600,2950159,Berlin,200
2,802,Clouds,scattered clouds,03d,-0.1257,51.5085,stations,9.56,6.63,7.92,...,1646761073,2,2019646,GB,1646721052,1646761923,0,2643743,London,200
3,803,Clouds,broken clouds,04d,-3.7026,40.4165,stations,11.23,9.95,9.5,...,1646760986,2,2007545,ES,1646721488,1646763204,3600,3117735,Madrid,200
4,800,Clear,clear sky,01d,-74.006,40.7143,stations,7.08,2.16,5.08,...,1646761018,2,2039034,US,1646738351,1646780080,-18000,5128581,New York,200
5,804,Clouds,overcast clouds,04n,37.6156,55.7522,stations,-5.15,-10.29,-6.03,...,1646761246,2,2018597,RU,1646712227,1646752636,10800,524901,Moscow,200
6,800,Clear,clear sky,01d,-74.006,40.7143,stations,7.08,2.16,5.08,...,1646761018,2,2039034,US,1646738351,1646780080,-18000,5128581,New York,200
7,803,Clouds,broken clouds,04n,32.8543,39.9199,stations,4.8,4.8,3.75,...,1646761221,2,267643,TR,1646712708,1646754439,10800,323786,Ankara,200
8,800,Clear,clear sky,01n,44.4009,33.3406,stations,15.95,14.85,15.95,...,1646761504,1,7597,IQ,1646709748,1646751858,10800,98182,Baghdad,200
9,804,Clouds,overcast clouds,04n,69.4167,34.5,stations,4.39,3.06,4.39,...,1646761495,NaN,NaN,AF,1646703782,1646745819,16200,1138957,Kabul,200


Fantastic! Now we have a DataFrame with current weather information for 15 locations. What's even better, we didn't have to write a lot of complicated code. How cool is that!?  

If we wanted to, we could now create a new weather table in our database and write the weather data into it. Guess what, this is exactly what you're going to do in the upcoming project. So take a break and then move on to the next challenge, good luck!

### Extra Credit 

If you have time and energy, we have an additional challenge for you.

Until now, you got to know two different methods to call an API:
1. Pass the general part of the URL and put the rest of the information into a dictionary and pass it as an additional argument.
<br>`r = requests.get(url_part, parameter)`
2. Put all parts of the URL as strings and pass the whole URL to the get function.
<br>`url_all = url_part + location + unit + api_key`
<br>`r = requests.get(url_all)`

Why don't you apply the first method (passing parameters to the get function) to the task in which you used the second method. <br> Can you do it?

In [44]:
# Extra Credit - Solution
# Set the url
url_part = 'http://api.openweathermap.org/data/2.5/weather?'

# Set locations
locations = ["Hamburg", 
             "Berlin", 
             "London", 
             "Madrid",
             "New York",
             "Moscow",
             "New York",
             "Ankara",
             "Baghdad",
             "Kabul",
             "Tokio",
             "Taipei",
             "Manila",
             "Auckland",
             "Seoul"]

# Set unit of measurement
unit = "&units=metric"

# Set API key (defined already)
api_key = '&appid=' + os.getenv('ow_key')

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])

for location in locations:
    parameters = {
        'appid': os.getenv('ow_key'),
        'q': location,
        'units': 'metric'}

    # Request data from url
    r = requests.get(url_part, parameters)
    
    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = r.json()
   
    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="weather", 
                                        meta=[["coord", "lon"], 
                                              ["coord", "lat"], 
                                              "base",
                                              ["main", "temp"],
                                              ["main", "feels_like"],
                                              ["main", "temp_min"],
                                              ["main", "temp_max"], 
                                              ["main", "pressure"], 
                                              ["main", "humidity"], 
                                              "visibility", 
                                              ["wind", "speed"], 
                                              ["wind", "deg"], 
                                              ["clouds", "all"], 
                                              "dt", 
                                              ["sys", "type"], 
                                              ["sys", "id"],
                                              ["sys", "country"],
                                              ["sys", "sunrise"],
                                              ["sys", "sunset"],
                                              "timezone",
                                              "id",
                                              "name",
                                              "cod"], 
                                        record_prefix="weather_",
                                        errors='ignore')
    
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df
    

,weather_id,weather_main,weather_description,weather_icon,coord_lon,coord_lat,base,main_temp,main_feels_like,main_temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod
0,800,Clear,clear sky,01n,10,53.55,stations,7.03,5.25,5.05,...,1646762077,1,1263,DE,1646718725,1646759391,3600,2911298,Hamburg,200
1,800,Clear,clear sky,01n,13.4105,52.5244,stations,5.67,4.09,2.86,...,1646761755,2,2011538,DE,1646717857,1646758622,3600,2950159,Berlin,200
2,802,Clouds,scattered clouds,03n,-0.1257,51.5085,stations,9.47,6.51,7.88,...,1646762050,2,2019646,GB,1646721052,1646761923,0,2643743,London,200
3,802,Clouds,scattered clouds,03d,-3.7026,40.4165,stations,11.24,9.93,8.51,...,1646761801,2,2007545,ES,1646721488,1646763204,3600,3117735,Madrid,200
4,803,Clouds,broken clouds,04d,-74.006,40.7143,stations,7.15,2.62,5.19,...,1646762192,2,2039034,US,1646738351,1646780080,-18000,5128581,New York,200
5,804,Clouds,overcast clouds,04n,37.6156,55.7522,stations,-5.05,-10.17,-6.03,...,1646761553,2,2018597,RU,1646712227,1646752636,10800,524901,Moscow,200
6,803,Clouds,broken clouds,04d,-74.006,40.7143,stations,7.15,2.62,5.19,...,1646762192,2,2039034,US,1646738351,1646780080,-18000,5128581,New York,200
7,804,Clouds,overcast clouds,04n,32.8543,39.9199,stations,4.56,2.61,4.56,...,1646761829,2,267643,TR,1646712708,1646754439,10800,323786,Ankara,200
8,800,Clear,clear sky,01n,44.4009,33.3406,stations,15.95,14.85,15.95,...,1646761440,1,7597,IQ,1646709748,1646751858,10800,98182,Baghdad,200
9,500,Rain,light rain,10n,69.4167,34.5,stations,4.39,3.06,4.39,...,1646762193,NaN,NaN,AF,1646703782,1646745819,16200,1138957,Kabul,200
